In [13]:
import os
import sys
sys.path.append(os.path.abspath("../ch04"))
from ipynb.fs.defs.gpt import GPTModel
import torch
import torch.nn as nn
folder_a_path = os.path.abspath('../ch04')  # Adjust the relative path
sys.path.append(folder_a_path)

GPT_CONFIG_124M = {
"vocab_size": 50257,
"context_length": 256, #A
"emb_dim": 768,
"n_heads": 12,
"n_layers": 12,
"drop_rate": 0.1, #B
"qkv_bias": False
}
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (layernorm1): LayerNorm()
      (layernorm2): LayerNorm()
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (attn): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (drop): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (layernorm1): LayerNorm()
      (layernorm2): LayerNorm()
    

Reimplementing tokenization and inverse operations

In [2]:
import tiktoken
from ipynb.fs.defs.gpt import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    input_tokens=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves youInstant surrounds receptor Importminus ay symbols audiences cheaply preventing


In [33]:
input1 = text_to_token_ids("every effort moves", tokenizer)
input2 = text_to_token_ids("I really like", tokenizer)

inputs = torch.stack([input1, input2], dim=1).squeeze(0)
print(inputs)

target1 = text_to_token_ids(" effort moves you", tokenizer)
target2 = text_to_token_ids(" really like chocolate", tokenizer)
print(target1.shape)
print(target2.shape)
targets = torch.stack([target1, target2], dim=1).squeeze(0)
print(targets)

tensor([[16833,  3626,  6100],
        [   40,  1107,   588]])
torch.Size([1, 3])
torch.Size([1, 3])
tensor([[ 3626,  6100,   345],
        [ 1107,   588, 11311]])


Compute loss    

In [5]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

print(inputs.shape)
print(targets.shape)


torch.Size([2, 3])
torch.Size([2, 3])


In [7]:
logits = model(inputs)
print(logits.shape)

torch.Size([2, 3, 50257])


In [15]:
logits_flattened = logits.flatten(0, 1)
targets_flattened = targets.flatten()
print(logits_flattened.shape)
print(targets_flattened.shape)


loss = nn.CrossEntropyLoss(logits_flattened, targets_flattened)
print(loss)

torch.Size([6, 50257])
torch.Size([6])


RuntimeError: Boolean value of Tensor with more than one value is ambiguous